In [1]:
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import minimize
import requests

N=6

def get_pos_from_url(address='http://doye.chem.ox.ac.uk/jon/structures/LJ/points/', N=7):
    url_address = address + str(N)
    data_str = requests.get(url_address).text
    return parse_url_text(data_str)    
    
def parse_url_text(data_str):
    x_array = []
    text = data_str.split('\n')
    for line in text:
        [x_array.append(float(i)) for i in line.split()]
    return np.array(x_array)

def dist(N_atoms):
    pos = get_pos_from_url(N = N_atoms)
    pos = np.reshape(pos,[N_atoms,3])
    #pos = 10*np.random.random(N_atoms*3)
    #pos = np.reshape(pos,[N_atoms,3])
    distance = cdist(pos,pos,'euclidean')
    iu1 = np.triu_indices(N_atoms)
    distance = distance[iu1]
    index = np.argwhere(distance==0)
    distance = np.delete(distance,index)
    return distance
    
def LJ(r):
    r6 = np.power(r,6)
    r12 = np.multiply(r6,r6)
    return 4*(1/r12 - 1/r6)
    
def Total_Energy(positions):

    E = LJ(positions)
    Energy = np.sum(E)
    return Energy

N_attempts = 50
f_values = []
x_values = []


def ground_state_potential_1(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='CG', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
    E=min(f_values)
    return E       

def ground_state_potential_2(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='Nelder-Mead', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
    E=min(f_values)
    return E       

    
def ground_state_potential_3(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='BFGS', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
    E=min(f_values)
    return E       


In [2]:
print('Method comparison for',N,'atoms:')
print('')
E1=ground_state_potential_1(N)
E2=ground_state_potential_2(N)
E3=ground_state_potential_3(N)
print('CG Method:')
print('Ground State Potential=',E1)
%timeit ground_state_potential_1(N)
print('')
print('Nelder-Mead Method:')
print('Ground State Potential=',E2)
%timeit ground_state_potential_2(N)
print('')
print('BFGS Method:')
print('Ground State Potential=',E3)
%timeit ground_state_potential_3(N)

Method comparison for 6 atoms:

CG Method:
Ground State Potential= -14.999999999888958
982 ms ± 9.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Nelder-Mead Method:
Ground State Potential= -14.999999999888958
5.7 s ± 16.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

BFGS Method:
Ground State Potential= -14.999999999999247
1.52 s ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
